# [(How to Write a (Lisp) Interpreter (in Python))](https://norvig.com/lispy.html)

## Scheme程序的语法和语义

一种语言的语法是将字符排列成正确的语句或表达式；语义是这些语句或表达式的含义。

例如，在数学表达式语言（以及许多编程语言）中，一加二的语法是“1+2”，语义是对两个数字进行加法运算，得到值3。

当我们确定一个表达式的值时，我们假设它是求值的；我们会说“1+2”的计算结果为3，并将其写为“1+2”⇒3。

Scheme语法与大多数其他编程语言不同。



| Java                                                         |      | Scheme                                                       |
| :----------------------------------------------------------- | ---- | ------------------------------------------------------------ |
|                                                              |      |                                                              |
| **if** (x.val() > 0)<br /> { **return** fn(A[i] + 3 * i,      **new** String[] {"one", "two"});} |      | (**if** (> (val x) 0)  (fn (+ (aref A i) (* 3 i))    (**quote** (one two))) |

Java有各种各样的语法约定（关键字、中缀运算符(infix operators)、三种括号、运算符优先级、点表示法、引号、逗号、分号），但是Scheme语法要简单得多：

- Scheme程序仅由*表达式*组成. 没有语句/表达式的区别.

- 数字 (e.g. `1`) ， 符号 (e.g. `A`) 都被成为*原子表达式* (*atomic expressions*); 他们不能被拆分. 它们与Java的对应部分相似，只是在Scheme中，+和>等运算符也是符号，它们的处理方式与 `A` 和 `fn`相同.

- 其他所有内容都是一个列表表达式

  一个 "(", 后跟零个或多个表达式，后跟一个 ")". 列表的第一个元素决定了它的含义:

  - 以关键字开头的列表, e.g. `(if ...)`,是一个*特殊形式* (*special form*); 其含义取决于关键字.
  - 以非关键字开头的列表, e.g. `(fn ...)`,是一个函数调用.

  

Scheme的优点是整个语言只需要5个关键字和8个句法形式(syntactic forms)。. 相比之下Python有33个关键字和 [110](https://docs.python.org/3/reference/grammar.html) 句法形式, Java 有50个关键字和[133](https://docs.oracle.com/javase/specs/jls/se7/html/jls-18.html) 个句法形式. 所有这些括号看起来都很吓人，但是Scheme语法具有简单和一致的优点。 (Some have joked that "Lisp" stands for "[***L**ots of **I**rritating **S**illy **P**arentheses*](http://www.google.com/search?q=Lots+of+Irritating+Silly+Parentheses)"; I think it stand for "[***L**isp **I**s **S**yntactically **P**ure*](http://www.google.com/search?hl=en&as_q=&as_epq=Lisp+Is+Syntactically+Pure)".)

在本页中，我们将讨论Scheme语言及其解释的所有要点（省略一些小细节），但我们将采取两个步骤来达到这一点，在定义近乎完整的Scheme语言之前，首先定义一个简化的语言。




## Language1:*Lispy*计算器

Lispy计算器是Scheme的一个子集，只使用五种语法形式（两种原子形式、两种特殊形式和过程调用）。Lispy Calculator允许你在典型的计算器上进行任何计算，只要你熟悉前缀标记( prefix notation)。你可以做两件在典型计算器语言中没有的事情：“if”表达式和新变量的定义。下面是一个示例程序，它使用公式πr2计算半径为10的圆的面积：

```scheme
(define r 10)
(* pi (* r r))
```

以下是所有允许表达式的表格：

| 表达式                                                       | 语法                        | 语义和例子                                                   |
| ------------------------------------------------------------ | --------------------------- | ------------------------------------------------------------ |
| [变量引用](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-7.html#%_sec_4.1.1) | *symbol*                    | 符号被解释为变量名；它的值就是变量的值<br /> Example: `r` ⇒ `10` (假设 `r` 之前被定义成10) |
| [常量字面(constant literal)](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-7.html#%_sec_4.1.2) | *number*                    | 一个数的计算结果是它自己.<br /> Examples: `12 ⇒ 12` *or* `-3.45e+6 ⇒ -3.45e+6` |
| [条件](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-7.html#%_sec_4.1.5) | `(if` *test conseq alt*`)`  | 计算 *test*；如果为true，则求值并返回*conseq*；否则为*alt*.<br />Example: `(if (> 10 20) (+ 1 1) (+ 3 3)) ⇒ 6` |
| [定义](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-8.html#%_sec_5.2) | `(define` *symbol* *exp*`)` | 定义一个新变量，并给它赋值表达式*exp*.<br /> Examples: `(define r 10)` |
| [过程调用](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-7.html#%_sec_4.1.3) | `(`*proc arg...*`)`         | 如果 *proc* 不是`if, define, `或 `quote` 之一，将其视为一个过程. 计算 *proc* 及其所有 *args*, 然后将该过程应用到*arg* 值列表. <br />Example: `(sqrt (* 2 8)) ⇒ 4.0` |

在该表的*语法*列中, *symbol* 必须是一个符号, *number* 必须是整数或浮点数, 其他斜体字可以是任何表达式.  *arg...* 标记是零个或多个 *arg*.



## 语言解释器做了什么？

语言解释器由两个部分构成:

1. **解析(Parsing)**:

    

   解析组件以字符序列的形式接收输入程序，根据语言的语法规则对其进行验证，并将程序转换为内部表示。在简单的解释器中，内部表示是一种树结构（通常称为抽象语法树）(abstract syntax tree) 它反映程序中语句或表达式的嵌套结构。

   

   在一个称为*编译器*的语言解释器程序中通常有一系列内部表示法，从抽象语法树开始，逐步发展到可由计算机直接执行的一系列指令. 

   

   Lispy解析器是用parse函数实现的.

   

2. **执行(Execution):** 然后根据语言的语义规则对内部表示进行处理，从而进行计算。Lispy的执行函数称为eval（请注意，它隐藏了Python同名的内置函数）.

>  program ➡ `parse` ➡ abstract-syntax-tree ➡ `eval` ➡ result

下面是一个简短的示例，说明我们希望parse和eval能够做什么（begin按顺序计算每个表达式并返回最后一个表达式）：

```bash
>> program = "(begin (define r 10) (* pi (* r r)))"

>>> parse(program)
['begin', ['define', 'r', 10], ['*', 'pi', ['*', 'r', 'r']]]

>>> eval(parse(program))
314.1592653589793
```



## 类型定义

让我们明确Scheme对象的表示：



In [7]:
Symbol = str              # A Scheme Symbol is implemented as a Python str
Number = (int, float)     # A Scheme Number is implemented as a Python int or float
Atom   = (Symbol, Number) # A Scheme Atom is a Symbol or Number
List   = list             # A Scheme List is implemented as a Python list
Exp    = (Atom, List)     # A Scheme expression is an Atom or List
Env    = dict             # A Scheme environment (defined below) 
                          # is a mapping of {variable: value}


## Parsing: `parse`, `tokenize` and `read_from_tokens`

传统上，语法分析分为两个部分: 词法分析(*lexical analysis*), 将输入的字符串分解成一系列 *tokens*, 句法分析(*syntactic analysis*), 将token组合成一个抽象的语法树. 

Lispy的token是括号、符号和数字. 词法分析有很多工具(比如 Mike Lesk and Eric Schmidt's [lex](http://dinosaur.compilertools.net/#lex)), 但是现在我们将使用一个非常简单的工具: Python的 `str.split`. 

函数 `tokenize`接受一个字符串作为输入；它在每个括号周围添加空格，然后调用 `str.split` 来得到token列表:

In [2]:
def tokenize(chars: str) -> list:
    "Convert a string of characters into a list of tokens."
    return chars.replace('(', ' ( ').replace(')', ' ) ').split()



```python
>>> program = "(begin (define r 10) (* pi (* r r)))"
>>> tokenize(program)
['(', 'begin', '(', 'define', 'r', '10', ')', '(', '*', 'pi', '(', '*', 'r', 'r', ')', ')', ')']
```

我们的函数 `parse` 将以一个程序的字符串作为输入, 调用 `tokenize` 来得到token列表, 然后调用 `read_from_tokens` 组合成一颗抽象语法树. 

`read_from_tokens` 查看第一个标记; 如果是一个 `')'` 说明语法错误. 如果是一个`'('`, 那么我们开始构建一个子表达式列表，直到找到匹配的 `')'`. 任何非圆括号token都必须是symbol或number.

 我们让Python来区分它们：对于每个非括号标记，首先尝试将其解释为int，然后解释为float，如果两者都不是，那么它一定是一个symbol. 



In [3]:
def parse(program: str) -> Exp:
    "Read a Scheme expression from a string."
    return read_from_tokens(tokenize(program))

def read_from_tokens(tokens: list) -> Exp:
    if len(tokens) == 0:
        raise SyntaxError('unexpected EOF')
    token = tokens.pop(0)
    if token == '(':
        L=[]
        while tokens[0] != ')':
            L.append(read_from_tokens(tokens))
        tokens.pop(0) # pop off ')'
        return L
    elif token == ')':
        raise SyntaxError('unexpected )')
    else:
        return atom(token)
    
def atom(token: str) -> Atom:
    try: return int(token)
    except ValueError:
        try: return float(token)
        except ValueError:
            return Symbol(token)

In [5]:
program = "(begin (define r 10) (* pi (* r r)))"

In [6]:
parse(program)

['begin', ['define', 'r', 10], ['*', 'pi', ['*', 'r', 'r']]]

> (begin exp1 exp2 ...): 
顺序执行表达式 exp1, exp2, ...，返回最后一个表达式的结果

## Environments

环境是从变量名到变量值的映射。默认情况下，eval将使用一个全局环境，其中包含一组标准函数的名称（如sqrt和max，以及运算符*）。可以使用表达式（define symbol value）通过用户定义的变量来扩展此环境。


> vars() 函数返回对象object的属性和属性值的字典对象。

In [4]:
class Data(object):
    def __init__(self,name,value):
        self.name = name
        self.value = value
class Value(object):
    def __init__(self,age=None,sex=None):
        self.age =age
        self.sex = sex
        
        
val_obj = vars(Value(age='63',sex='man'))
data_obj = vars(Data('Norvig',val_obj ))
print(data_obj )

{'name': 'Norvig', 'value': {'age': '63', 'sex': 'man'}}


In [8]:
import math
import operator as op

def standard_env() -> Env:
    "An environment with some Scheme standard procedures."
    env = Env()
    env.update(vars(math)) # sin, cos, sqrt, pi, ...
    env.update({
        '+':op.add, '-':op.sub, '*': op.mul, '/':op.truediv,
        '>':op.gt, '<':op.lt, '>=':op.ge, '<=':op.le, '=':op.eq,
        'abs':abs,
        'append':op.add,
        'apply':lambda proc, args: proc(*args),
        'begin':lambda *x:x[-1],
        'car':  lambda x:x[0],
        'cdr':  lambda x:x[1:],
        'cons': lambda x,y:[x]+y,
        'eq?':  op.is_,
        'expt': pow,
        'equal?': op.eq,
        'length': len,
        'list': lambda *x:List(x),
        'list?': lambda x: isinstance(x, List),
        'map': map,
        'max': max,
        'min': min,
        'not': op.not_,
        'null?': lambda x: x==[],
        'number?': lambda x: isinstance(x, Number),
        'print': print,
        'procedure?': callable,
        'round': round,
        'symbol?': lambda x: isinstance(x, Symbol)
    })
    return env

global_env=standard_env()

## Evaluation: `eval`

我们来实现 `eval`.复习一下表达式:




| 表达式                                                       | 语法                        | 语义和例子                                                   |
| ------------------------------------------------------------ | --------------------------- | ------------------------------------------------------------ |
| [变量引用](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-7.html#%_sec_4.1.1) | *symbol*                    | 符号被解释为变量名；它的值就是变量的值<br /> Example: `r` ⇒ `10` (假设 `r` 之前被定义成10) |
| [常量字面(constant literal)](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-7.html#%_sec_4.1.2) | *number*                    | 一个数的计算结果是它自己.<br /> Examples: `12 ⇒ 12` *or* `-3.45e+6 ⇒ -3.45e+6` |
| [条件](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-7.html#%_sec_4.1.5) | `(if` *test conseq alt*`)`  | 计算 *test*；如果为true，则求值并返回*conseq*；否则为*alt*.<br />Example: `(if (> 10 20) (+ 1 1) (+ 3 3)) ⇒ 6` |
| [定义](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-8.html#%_sec_5.2) | `(define` *symbol* *exp*`)` | 定义一个新变量，并给它赋值表达式*exp*.<br /> Examples: `(define r 10)` |
| [过程调用](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-7.html#%_sec_4.1.3) | `(`*proc arg...*`)`         | 如果 *proc* 不是`if, define, `或 `quote` 之一，将其视为一个过程. 计算 *proc* 及其所有 *args*, 然后将该过程应用到*arg* 值列表. <br />Example: `(sqrt (* 2 8)) ⇒ 4.0` |



 `eval`的代码完全遵循上述表格:

In [5]:
def eval(x: Exp, env=global_env) -> Exp:
    "Evaluate an expression in an environment."
    if isinstance(x, Symbol):        # variable reference
        return env[x]
    elif isinstance(x, Number):      # constant number
        return x
    elif x[0] == 'if':               # conditional
        (_, test, conseq, alt) = x
        exp = (conseq if eval(test, env) else alt)
        return eval(exp, env)
    elif x[0] == 'define':           # definition
        (_, symbol, exp) = x
        env[symbol] = eval(exp, env)
    else:                            # procedure call
        proc = eval(x[0], env)
        args = [eval(arg, env) for arg in x[1:]]
        return proc(*args)

In [61]:
eval(parse("(begin (define r 10) (* pi (* r r)))"))

314.1592653589793

## 交互: A REPL

一直输入 `eval(parse("..."))` 很无聊. Lisp的一个伟大遗产是读-求值-打印的交互式循环: 程序员可以输入一个表达式，并看到它立即被读取、求值和打印, 而不必经历漫长的构建/编译/运行周期. 让我们定义函数`repl`（代表read eval print loop）和函数`schemestr`(它返回一个表示Scheme对象的字符串).

In [12]:
def repl(prompt='lis.py> '):
    "A prompt-read-eval-print loop."
    while True:
        val = eval(parse(input(prompt)))
        if val is not None:
            print(schemestr(val))

def schemestr(exp):
    "Convert a Python object back into a Scheme-readable string."
    if isinstance(exp, List):
        return '(' + ' '.join(map(schemestr, exp)) + ')'
    else:
        return str(exp)
    

In [73]:
exp=['begin', ['define', 'r', 10], ['*', 'pi', ['*', 'r', 'r']]]
schemestr(exp)

'(begin (define r 10) (* pi (* r r)))'

In [ ]:
repl()

## Language 2: 完整 Lispy

现在，我们将用三种新的特殊形式来扩展我们的语言，给我们一个更接近完整的Scheme子集：



| Expression                                                   | Syntax                             | Semantics and Example                                        |
| ------------------------------------------------------------ | ---------------------------------- | ------------------------------------------------------------ |
| [quotation](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-7.html#%_sec_4.1.2) | `(quote `*exp*`)`                  | 返回 *exp* 字面; 不计算. Example: `(quote (+ 1 2)) ⇒ (+ 1 2)` |
| [assignment](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-7.html#%_sec_4.1.6) | `(set!` *symbol exp*`)`            | 计算 *exp* 并将值赋给 *symbol*, 这个*symbol*必须是之前定义过的 (使用过 `define` 或是一个封闭过程的参数). Example: `(set! r2 (* r r))` |
| [procedure](http://www.schemers.org/Documents/Standards/R5RS/HTML/r5rs-Z-H-7.html#%_sec_4.1.4) | `(lambda (`*symbol...*`)` *exp*`)` | 创建一个带参数的过程 parameter(s) 由 *symbol...* 和 *exp* 构成. Example: `(lambda (r) (* pi (* r r)))` |



`lambda` ([lambda calculus](http://en.wikipedia.org/wiki/Lambda_calculus)) 创建一个过程。我们希望该过程能像这样工作:
```scheme
lis.py> (define circle-area (lambda (r) (* pi (* r r)))
lis.py> (circle-area (+ 5 5))
314.159265359
```

两个步骤. 
1.  `lambda` 创建一个过程, 该过程引用全局变量 `pi` 和 `*`, 接受单参数 `r`. 过程用作新变量 `circle-area`的值. 
2.  我们定义了过程 `circle-area`, 我们用10作为参数调用它. 我们希望 `r` 但是在全局变量中将 `r` 设置成10是行不通的. 假如我们之前将 `r` 做了其他用途? 我们不希望因为调用 `circle-area` 而改变该值. 相反，我们希望它是 *局部* 变量 `r` 这样就不必担心会干扰任何碰巧同名的全局变量. 调用过程的进程引入了这些新的局部变量，将函数参数列表中的每个符号绑定到函数调用的参数列表中的相应值。

## Redefining `Env` as a Class

为了处理局部变量，我们把 `Env` 定义为 `dict` 的子类. 当我们求值 `(circle-area (+ 5 5))`, 我们获取过程体, `(* pi (* r r))`, 并在一个以 `r` 为唯一局部变量(值为10), 且全局环境为 "outer" 环境的环境中对其求值; 在这个环境中，我们可以找到 `*` and `pi`. 

当我们在这样一个嵌套的环境中查找变量时，我们首先查看最里面的一级，但是如果在那里找不到变量名，我们就转到下一级的外部

过程和环境联系紧密，所以一起定义它们



In [9]:
class Env(dict):
    "An environment: a dict of {'var': val} pairs, with an outer Env."
    def __init__(self, parms=(), args=(), outer=None):
        self.update(zip(parms, args))    # 绑定形参和实参值
        self.outer=outer
    def find(self, var):
        "Find the innermost Env where var appears."
        return self if (var in self) else self.outer.find(var)
    
class Procedure():
    "A user-defined Scheme procedure."
    def __init__(self, parms, body, env):
        self.parms, self.body, self.env = parms, body, env
    def __call__(self, *args):
        return eval(self.body, Env(self.parms, args, self.env))
    
global_env = standard_env()

我们看到，每个过程都有三个组件：一个参数名列表、一个主体表达式和一个环境，它告诉我们可以从主体访问哪些其他变量。对于在顶层定义的过程，这将是全局环境，但过程也可能引用*定义*它的环境（而不是*调用*它的环境）的局部变量。

环境是dict的一个子类，因此它拥有dict拥有的所有方法。此外，还有两种方法：构造函数通过获取参数名列表和相应的参数值列表来构建新环境，并创建一个新环境，该环境将{variable:value}对作为内部部分，并引用给定的外部环境。find方法用于为变量查找合适的环境：内部环境或外部环境。



为了了解这些是如何结合在一起的，这里是eval的新定义。注意，变量引用的子句已经更改：我们现在必须调用环境查找（x） 找到变量x存在的级别；然后我们可以从该级别获取x的值。（define的子句没有更改，因为define总是向最里面的环境添加一个新变量。）有两个新子句：对于set!，我们找到变量存在的环境级别并将其设置为新值。对于lambda，我们使用给定的参数列表、主体和环境创建一个新的过程对象。

In [10]:
def eval(x, env=global_env):
    "Evaluate an expression in an environment."
    if isinstance(x, Symbol):    # variable reference
        return env.find(x)[x]
    elif not isinstance(x, List):# constant 
        return x   
    op, *args = x       
    if op == 'quote':            # quotation
        return args[0]
    elif op == 'if':             # conditional
        (test, conseq, alt) = args
        exp = (conseq if eval(test, env) else alt)
        return eval(exp, env)
    elif op == 'define':         # definition
        (symbol, exp) = args
        env[symbol] = eval(exp, env)
    elif op == 'set!':           # assignment
        (symbol, exp) = args
        env.find(symbol)[symbol] = eval(exp, env)
    elif op == 'lambda':         # procedure
        (parms, body) = args
        return Procedure(parms, body, env)
    else:                        # procedure call
        proc = eval(op, env)
        vals = [eval(arg, env) for arg in args]
        return proc(*vals)